In [ ]:
import os
import csv
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Function to normalize landmarks relative to the wrist (index 0)
def normalize_landmarks(landmarks):
    if not landmarks or len(landmarks) != 21:
        return None

    base_x, base_y = landmarks[0][0], landmarks[0][1]
    
    normalized_landmarks = [(l[0] - base_x, l[1] - base_y) for l in landmarks]
    return normalized_landmarks

# Function to extract landmarks (only x and y) from an image
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to read image {image_path}")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
            
            # Ensure exactly 21 landmarks are extracted
            if len(landmarks) == 21:
                return normalize_landmarks(landmarks)
    
    return None  # No hands detected

# Function to process all images in a directory
def process_directory(dataset_dir, output_csv):
    dataset_dir = os.path.abspath(dataset_dir)  # Ensure absolute path
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header: Label, Landmarks (x1, y1, x2, y2, ..., x21, y21)
        header = ['Label'] + [f'x{i+1}' for i in range(21)] + [f'y{i+1}' for i in range(21)]
        writer.writerow(header)

        for label in os.listdir(dataset_dir):
            label_dir = os.path.join(dataset_dir, label)
            if os.path.isdir(label_dir):
                for image_name in os.listdir(label_dir):
                    if image_name.startswith('.'):  # Skip hidden/system files
                        continue

                    image_path = os.path.join(label_dir, image_name)
                    landmarks = extract_landmarks(image_path)

                    if landmarks:
                        flattened_landmarks = [coord for point in landmarks for coord in point]
                        writer.writerow([label] + flattened_landmarks)
                    else:
                        print(f"Skipping {image_path}, no hands detected.")

# Main function
def main():
    dataset_dir = 'RGB ArSL dataset'  # Path to dataset
    output_csv = 'landmarks_normalized.csv'  # Output CSV
    process_directory(dataset_dir, output_csv)

if __name__ == '__main__':
    main()


Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_114.jpeg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_126.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_132.jpeg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_136.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_16.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_162.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_165.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_169.jpg, no hands detected.
Skipping c:\Users\Abdulrhman Hagras\Desktop\Arabic_Resources\RGB ArSL dataset\Ain\Ain_1